<a href="https://colab.research.google.com/github/DikshantBadawadagi/Image-Binary-Matrix/blob/main/BitPlaneComplexitySegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from PIL import Image
import random

def logistic_map_2d(x0, y0, r, shape):
    x, y = x0, y0
    sequence = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x = r * x * (1 - x)
            y = r * y * (1 - y)
            sequence[i, j] = (x + y) / 2
    return sequence

def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    array = np.clip(array, 0, 255).astype(np.uint8)
    image = Image.fromarray(array)
    image.save(output_path)

def arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (x + y) % width
                new_y = (x + 2*y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def inverse_arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (2*x - y) % width
                new_y = (-x + y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def encrypt_image(image_array, key, arnold_iterations):
    encrypted = arnold_transform(image_array, arnold_iterations)
    encrypted = encrypted.astype(np.int16)
    if len(image_array.shape) == 3:  # Color image
        for c in range(image_array.shape[2]):
            encrypted[:,:,c] = (encrypted[:,:,c] + (key * 256).astype(np.int16)) % 256
    else:  # Grayscale image
        encrypted = (encrypted + (key * 256).astype(np.int16)) % 256
    return encrypted.astype(np.uint8)

def decrypt_image(encrypted_array, key, arnold_iterations):
    decrypted = encrypted_array.astype(np.int16)
    if len(encrypted_array.shape) == 3:  # Color image
        for c in range(encrypted_array.shape[2]):
            decrypted[:,:,c] = (decrypted[:,:,c] - (key * 256).astype(np.int16)) % 256
    else:  # Grayscale image
        decrypted = (decrypted - (key * 256).astype(np.int16)) % 256
    decrypted = inverse_arnold_transform(decrypted.astype(np.uint8), arnold_iterations)
    return decrypted.astype(np.uint8)

def encrypt_image_file(input_path, output_path):
    x0 = random.uniform(0, 1)
    y0 = random.uniform(0, 1)
    r = random.uniform(3.8, 4.0)
    arnold_iterations = random.randint(1, 10)
    image_array = image_to_array(input_path)
    key = logistic_map_2d(x0, y0, r, image_array.shape[:2])
    encrypted_array = encrypt_image(image_array, key, arnold_iterations)
    array_to_image(encrypted_array, output_path)
    return x0, y0, r, arnold_iterations

def decrypt_image_file(encrypted_path, output_path, x0, y0, r, arnold_iterations):
    encrypted_array = image_to_array(encrypted_path)
    key = logistic_map_2d(x0, y0, r, encrypted_array.shape[:2])
    decrypted_array = decrypt_image(encrypted_array, key, arnold_iterations)
    array_to_image(decrypted_array, output_path)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encryption
x0, y0, r, arnold_iterations = encrypt_image_file(input_image_path, encrypted_image_path)
print(f"Encryption parameters: x0={x0}, y0={y0}, r={r}, arnold_iterations={arnold_iterations}")

# Decryption
decrypt_image_file(encrypted_image_path, decrypted_image_path, x0, y0, r, arnold_iterations)
print("Decryption complete.")

# Debugging
original_image = image_to_array(input_image_path)
encrypted_image = image_to_array(encrypted_image_path)
decrypted_image = image_to_array(decrypted_image_path)

print("Original image shape:", original_image.shape)
print("Encrypted image shape:", encrypted_image.shape)
print("Decrypted image shape:", decrypted_image.shape)

print("Original image dtype:", original_image.dtype)
print("Encrypted image dtype:", encrypted_image.dtype)
print("Decrypted image dtype:", decrypted_image.dtype)

print("Original image sample:", original_image[0, 0])
print("Encrypted image sample:", encrypted_image[0, 0])
print("Decrypted image sample:", decrypted_image[0, 0])

difference = np.abs(original_image.astype(np.float64) - decrypted_image.astype(np.float64))
mean_difference = np.mean(difference)
print(f"Mean difference between original and decrypted images: {mean_difference}")

max_difference = np.max(difference)
print(f"Maximum difference between original and decrypted images: {max_difference}")

# Check if any pixel values are different
if np.array_equal(original_image, decrypted_image):
    print("The decrypted image is identical to the original image.")
else:
    print("The decrypted image differs from the original image.")
    different_pixels = np.sum(original_image != decrypted_image)
    print(f"Number of different pixels: {different_pixels}")

Encryption parameters: x0=0.45873957524167297, y0=0.5292915928689771, r=3.8180785198327727, arnold_iterations=3
Decryption complete.
Original image shape: (225, 225, 3)
Encrypted image shape: (225, 225, 3)
Decrypted image shape: (225, 225, 3)
Original image dtype: uint8
Encrypted image dtype: uint8
Decrypted image dtype: uint8
Original image sample: [166 160 110]
Encrypted image sample: [153 147  97]
Decrypted image sample: [166 160 110]
Mean difference between original and decrypted images: 0.0
Maximum difference between original and decrypted images: 0.0
The decrypted image is identical to the original image.


In [8]:
import numpy as np
from PIL import Image
import random

def logistic_map_2d(x0, y0, r, shape):
    x, y = x0, y0
    sequence = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            x = r * x * (1 - x)
            y = r * y * (1 - y)
            sequence[i, j] = (x + y) / 2
    return sequence

def image_to_array(image_path):
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    array = np.clip(array, 0, 255).astype(np.uint8)
    image = Image.fromarray(array)
    image.save(output_path)

def arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (x + y) % width
                new_y = (x + 2*y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

def inverse_arnold_transform(image, num_iterations):
    height, width = image.shape[:2]
    result = np.zeros_like(image)

    for _ in range(num_iterations):
        for y in range(height):
            for x in range(width):
                new_x = (2*x - y) % width
                new_y = (-x + y) % height
                result[new_y, new_x] = image[y, x]
        image = result.copy()

    return result

# DNA coding functions
def int_to_dna(n):
    dna_map = {0: 'A', 1: 'T', 2: 'C', 3: 'G'}
    return dna_map[n]

def dna_to_int(dna):
    dna_map = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    return dna_map[dna]

def pixel_to_dna(pixel):
    return ''.join(int_to_dna((pixel >> i) & 3) for i in (6, 4, 2, 0))

def dna_to_pixel(dna):
    return sum(dna_to_int(dna[i]) << (6 - 2*i) for i in range(4))

def dna_encode(array):
    return np.vectorize(pixel_to_dna)(array)

def dna_decode(dna_array):
    return np.vectorize(dna_to_pixel)(dna_array)

def dna_xor(dna1, dna2):
    return ''.join(int_to_dna(dna_to_int(a) ^ dna_to_int(b)) for a, b in zip(dna1, dna2))

def generate_key(shape):
    return np.random.randint(0, 256, shape, dtype=np.uint8)

def encrypt_image(image_array, key, arnold_iterations):
    # Arnold transform
    encrypted = arnold_transform(image_array, arnold_iterations)
    encrypted = encrypted.astype(np.int16)

    # DNA encryption
    if len(encrypted.shape) == 3:  # Color image
        encrypted_dna = np.empty_like(encrypted, dtype=object)
        for c in range(encrypted.shape[2]):
            dna_image = dna_encode(encrypted[:,:,c])
            dna_key = dna_encode(key)
            encrypted_dna[:,:,c] = np.frompyfunc(dna_xor, 2, 1)(dna_image, dna_key)
        encrypted = np.apply_along_axis(lambda x: dna_decode(x.astype(str)), 2, encrypted_dna)
    else:  # Grayscale image
        dna_image = dna_encode(encrypted)
        dna_key = dna_encode(key)
        encrypted_dna = np.frompyfunc(dna_xor, 2, 1)(dna_image, dna_key)
        encrypted = dna_decode(encrypted_dna)

    return encrypted.astype(np.uint8)

def decrypt_image(encrypted_array, key, arnold_iterations):
    # DNA decryption
    if len(encrypted_array.shape) == 3:  # Color image
        decrypted = np.empty_like(encrypted_array, dtype=object)
        for c in range(encrypted_array.shape[2]):
            dna_encrypted = dna_encode(encrypted_array[:,:,c])
            dna_key = dna_encode(key)
            decrypted[:,:,c] = np.frompyfunc(dna_xor, 2, 1)(dna_encrypted, dna_key)
        decrypted = np.apply_along_axis(lambda x: dna_decode(x.astype(str)), 2, decrypted)
    else:  # Grayscale image
        dna_encrypted = dna_encode(encrypted_array)
        dna_key = dna_encode(key)
        decrypted_dna = np.frompyfunc(dna_xor, 2, 1)(dna_encrypted, dna_key)
        decrypted = dna_decode(decrypted_dna)

    # Inverse Arnold transform
    decrypted = inverse_arnold_transform(decrypted.astype(np.uint8), arnold_iterations)
    return decrypted.astype(np.uint8)

def encrypt_image_file(input_path, output_path):
    x0 = random.uniform(0, 1)
    y0 = random.uniform(0, 1)
    r = random.uniform(3.8, 4.0)
    arnold_iterations = random.randint(1, 10)
    image_array = image_to_array(input_path)
    key = logistic_map_2d(x0, y0, r, image_array.shape[:2])
    key_array = (key * 255).astype(np.uint8)
    encrypted_array = encrypt_image(image_array, key_array, arnold_iterations)
    array_to_image(encrypted_array, output_path)
    return x0, y0, r, arnold_iterations


def decrypt_image_file(encrypted_path, output_path, x0, y0, r, arnold_iterations):
    encrypted_array = image_to_array(encrypted_path)
    key = logistic_map_2d(x0, y0, r, encrypted_array.shape[:2])
    key_array = (key * 255).astype(np.uint8)
    decrypted_array = decrypt_image(encrypted_array, key_array, arnold_iterations)
    array_to_image(decrypted_array, output_path)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encryption
x0, y0, r, arnold_iterations = encrypt_image_file(input_image_path, encrypted_image_path)
print(f"Encryption parameters: x0={x0}, y0={y0}, r={r}, arnold_iterations={arnold_iterations}")

# Decryption
decrypt_image_file(encrypted_image_path, decrypted_image_path, x0, y0, r, arnold_iterations)
print("Decryption complete.")

# Debugging
original_image = image_to_array(input_image_path)
encrypted_image = image_to_array(encrypted_image_path)
decrypted_image = image_to_array(decrypted_image_path)

print("Original image shape:", original_image.shape)
print("Encrypted image shape:", encrypted_image.shape)
print("Decrypted image shape:", decrypted_image.shape)

print("Original image dtype:", original_image.dtype)
print("Encrypted image dtype:", encrypted_image.dtype)
print("Decrypted image dtype:", decrypted_image.dtype)

print("Original image sample:", original_image[0, 0])
print("Encrypted image sample:", encrypted_image[0, 0])
print("Decrypted image sample:", decrypted_image[0, 0])

difference = np.abs(original_image.astype(np.float64) - decrypted_image.astype(np.float64))
mean_difference = np.mean(difference)
print(f"Mean difference between original and decrypted images: {mean_difference}")

max_difference = np.max(difference)
print(f"Maximum difference between original and decrypted images: {max_difference}")

# Check if any pixel values are different
if np.array_equal(original_image, decrypted_image):
    print("The decrypted image is identical to the original image.")
else:
    print("The decrypted image differs from the original image.")
    different_pixels = np.sum(original_image != decrypted_image)
    print(f"Number of different pixels: {different_pixels}")

Encryption parameters: x0=0.39260454132086176, y0=0.035457733728950336, r=3.9462461604091486, arnold_iterations=3
Decryption complete.
Original image shape: (225, 225, 3)
Encrypted image shape: (225, 225, 3)
Decrypted image shape: (225, 225, 3)
Original image dtype: uint8
Encrypted image dtype: uint8
Decrypted image dtype: uint8
Original image sample: [166 160 110]
Encrypted image sample: [ 47  41 231]
Decrypted image sample: [166 160 110]
Mean difference between original and decrypted images: 0.0
Maximum difference between original and decrypted images: 0.0
The decrypted image is identical to the original image.


BIT PLANE COMPLEXITY SEGEMENTATION

In [19]:
import numpy as np
from PIL import Image

def image_to_array(image_path):
    """Convert image to NumPy array."""
    image = Image.open(image_path)
    return np.array(image)

def array_to_image(array, output_path):
    """Convert NumPy array to image and save."""
    print(f"Debug: Array shape before conversion: {array.shape}")
    print(f"Debug: Array dtype: {array.dtype}")

    if array.ndim == 4 and array.shape[3] == 1:
        array = array.squeeze(axis=3)

    if array.ndim == 3 and array.shape[2] == 3:
        image = Image.fromarray(array.astype(np.uint8), mode='RGB')
    elif array.ndim == 2:
        image = Image.fromarray(array.astype(np.uint8), mode='L')
    else:
        raise ValueError(f"Unsupported array shape for image conversion: {array.shape}")
    image.save(output_path)

def split_into_bit_planes(image_array):
    """Split the image array into bit-planes."""
    print(f"Debug: Image array shape before splitting: {image_array.shape}")

    # Ensure the array is 3D (add channel dimension if grayscale)
    if image_array.ndim == 2:
        image_array = image_array[:, :, np.newaxis]

    # Split each color channel into bit planes
    height, width, channels = image_array.shape
    bit_planes = np.unpackbits(image_array, axis=2).reshape(height, width, channels, 8)

    print(f"Debug: Bit planes shape: {bit_planes.shape}")
    return bit_planes

def reconstruct_from_bit_planes(bit_planes):
    """Reconstruct image from bit-planes."""
    print(f"Debug: Bit planes shape before reconstruction: {bit_planes.shape}")

    reconstructed = np.packbits(bit_planes, axis=-1)
    print(f"Debug: Reconstructed shape: {reconstructed.shape}")

    return reconstructed.squeeze(axis=-1)

def encrypt_decrypt_bit_planes(bit_planes, key):
    """Encrypt or decrypt bit-planes by XORing with a key."""
    return bit_planes ^ key[:, :, np.newaxis, np.newaxis]

def generate_random_key(seed, shape):
    """Generate a random key for XOR encryption using a seed."""
    np.random.seed(seed)
    return np.random.randint(0, 2, shape, dtype=np.uint8)

def process_image_bpcs(image_path, output_path, seed=42, encrypt=True):
    """Encrypt or decrypt an image using Bit-Plane Complexity Segmentation (BPCS)."""
    image_array = image_to_array(image_path)
    print(f"Debug: Original image shape: {image_array.shape}")

    bit_planes = split_into_bit_planes(image_array)

    # Generate a random key for encryption based on seed
    key_shape = (bit_planes.shape[0], bit_planes.shape[1])
    key = generate_random_key(seed, key_shape)

    # Encrypt or decrypt the bit-planes
    processed_bit_planes = encrypt_decrypt_bit_planes(bit_planes, key)

    # Reconstruct the processed image
    processed_image = reconstruct_from_bit_planes(processed_bit_planes)
    print(f"Debug: Processed image shape: {processed_image.shape}")

    array_to_image(processed_image, output_path)

def encrypt_image_bpcs(image_path, output_path, seed=42):
    """Encrypt an image using BPCS."""
    process_image_bpcs(image_path, output_path, seed, encrypt=True)

def decrypt_image_bpcs(encrypted_path, output_path, seed=42):
    """Decrypt an image using BPCS."""
    process_image_bpcs(encrypted_path, output_path, seed, encrypt=False)

# Example Usage
input_image_path = '/content/download.jpg'
encrypted_image_path = '/content/encrypted_image.png'
decrypted_image_path = '/content/decrypted_image.png'

# Encrypt the image
encrypt_image_bpcs(input_image_path, encrypted_image_path)

# Decrypt the image
decrypt_image_bpcs(encrypted_image_path, decrypted_image_path)

Debug: Original image shape: (225, 225, 3)
Debug: Image array shape before splitting: (225, 225, 3)
Debug: Bit planes shape: (225, 225, 3, 8)
Debug: Bit planes shape before reconstruction: (225, 225, 3, 8)
Debug: Reconstructed shape: (225, 225, 3, 1)
Debug: Processed image shape: (225, 225, 3)
Debug: Array shape before conversion: (225, 225, 3)
Debug: Array dtype: uint8
Debug: Original image shape: (225, 225, 3)
Debug: Image array shape before splitting: (225, 225, 3)
Debug: Bit planes shape: (225, 225, 3, 8)
Debug: Bit planes shape before reconstruction: (225, 225, 3, 8)
Debug: Reconstructed shape: (225, 225, 3, 1)
Debug: Processed image shape: (225, 225, 3)
Debug: Array shape before conversion: (225, 225, 3)
Debug: Array dtype: uint8


BITPLANE combined with LOGICTIC 2D maps combined with DNA encoding
